##  Метод оцифровки pdf таблиц

###### Импорт модулей и библиотек

In [54]:
import tabula
import pandas as pd
import numpy as np

###### Производим оцифровку таблиц

In [55]:
# Функция находит строки без цифр, то есть шапку и возращает индекс последней строки шапки
def get_slice_header_index(df):
    
    for i, row in df.iterrows():
        list_val = list(map(str, row.dropna().values.tolist()))

        last_val = list_val[-1]
        if ' ' in last_val:
            last_val = last_val.split(' ')[-1]
        line_val = ' '.join(list_val)
        try:
            last_val = int(last_val)
            list_val_correct = list(map(int, line_val.split(' ')))
        except ValueError:
            continue

        if list_val_correct == list(range(1, last_val+1)):
            row_to_slice = i
            return row_to_slice

In [56]:
# Функция обрезает датафрейм по годам и возвращает список датафреймов для обработки
def df_with_year(df):
    
    l = []
    df_lst = []
    slc = get_slice_header_index(df) #передаем индекс для слайса
    df = df[slc:] #формируем датафрейм без шапки
    for i in list(df.iloc[:,0]): #ищем строки с обозначением года
        if '20' in i:
            l.append(df.loc[df.iloc[:,0] == i].index.tolist())
    if len(l) == 2: #для строки с годом выполняем распаковку сформированного списка
        l = [*l[0], *l[1]]
        df_lst.append(df[(l[0]-slc):(l[1]-slc)]) #добавляем в список датафреймов обрезанный датафрейм по году
        df_lst.append(df[l[1]-slc:])
    else:
        l = [*l[0]]
        df_lst.append(df[(l[0]-slc):])
    return df_lst

In [57]:
# Функция добавляет в датафрейм столбец с месяцем и годом
def create_df(df_year):
    year = df_year.iat[0,0][:-4] #находим значение года
    d = {}
    new_value = []
    date = []
    df_year = df_year[1:]
    for i in range(len(df_year)):
        values = []
        for k in range(len(df_year.columns)):
            values.append(df_year.iloc[i][k])
            d[i] = values
    for i in range(len(d)):
        for el in d[i]:
            new_value.append(el.split())
        d[i]=new_value
        new_value = []
    for key in d.keys():
        d[key] = [x for l in d[key] for x in l]
    df = pd.DataFrame(d).T
    for i in df[0]:
        date.append(i+' '+year) 
    df[0] = date
    df_year = df
    return(df_year)

In [58]:
# Функция выполняет оцифровку таблиц пдф и создает эксель таблицу
def result_table(pages):
    mytable = tabula.read_pdf(path_to_file, pages = pages) 
    dfs = []
    for table in mytable:
        df = pd.DataFrame(table)
        for i in df_with_year(df):
            dfs.append(create_df(i))
    res = pd.concat(dfs)
    res.replace(r'—', np.nan, regex=True, inplace=True)
    res.replace(r',', '.', regex=True, inplace=True)
    for column in res:
        if column != 0:
            res[column] = res[column].astype(float)
    res.to_excel(f'./Таблицы/{table_name}')

In [59]:
def main(pages_to_convert):
    
    for pages in pages_to_convert:
        table_name = 'table on page(s) ' # Формируем название файла
        if type(pages) == list:
            table_name += '-'.join([str(page) for page in pages]) + '.xlsx'
        else:
            table_name += str(pages)
        table_name
        
        result_table(pages) 
        print(f'{table_name} сформирована')

In [60]:
pages_to_convert = [79, [127,128], [129,130]] # Выбираем необходимые страницы
path_to_file = './Исходный файл/Bbs2110r.pdf' # Прописываем путь к файлу
main(pages_to_convert)

table on page(s) 79 сформирована
table on page(s) 127-128.xlsx сформирована
table on page(s) 129-130.xlsx сформирована
